In [ ]:
import pandas as pd
from training_claude import load_agent, evaluate_agent
from rl_env import FlexSimEnv
from eval import evaluate_heuristic
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
import time
from utils import create_field_from_list_column
from params import ON_TIME_BOUNDS, N_STOPS, MAX_TIME_HOURS, TRIP_WEIGHT, HEURISTIC_BETA
# resolution to 100 dpi
mpl.rcParams['figure.dpi'] = 120

import matplotlib.font_manager as fm
fm.addfont = fm.FontProperties(fname='fonts/Montserrat.zip')


In [ ]:
main_history = {
    'pax': [],
    'vehicles': [],
    'idle': [],
}
N_EPISODES = 30
RANDOM_SEED = 1

# slope scenarios
alpha = 1.2
demand = 'peak'

In [ ]:
scenario_name = f'heuristic_demand'
np.random.seed(RANDOM_SEED)
tmp_history = evaluate_heuristic(
    alpha=alpha, beta=HEURISTIC_BETA, n_episodes=N_EPISODES, 
    output_history=True,
    scenario_name=scenario_name, demand_scenario=demand)
for ky in tmp_history:
    if ky not in main_history:
        main_history[ky] = []
    tmp_history[ky]['demand'] = demand
    main_history[ky].append(tmp_history[ky])

## trash

In [2]:
env = EnvironmentManager(reward_weight=TRIP_WEIGHT)
env.start_vehicles()
env.route.load_all_pax(demand_scenario='off_peak')

observation, reward, done, terminated, info = env.step(action=None)
while not terminated:
    if not done:
        action = get_action(
            'DRD', observation, 
            minimum_requests_slope=1.0, base_minimum_requests=1.5) # ideal slope
    else:
        action = None
    observation, reward, done, terminated, info = env.step(action=action)
history = env.get_history()

In [7]:
from eval import evaluate_heuristic

# function to receive a float and return a string with '_' instead of '.'
def float_to_str(f):
    return str(f).replace('.', '_')

main_history = {
    'pax': [],
    'vehicles': [],
    'idle': [],
}
N_EPISODES = 30
RANDOM_SEED = 1

# slope scenarios
alphas = [1.0, 1.2, 1.5, 1.8]
# alphas = [1.2,]

main_history = {
    'pax': [],
    # 'state': [],
    'vehicles': [],
    'idle': [],
}
N_EPISODES = 30
RANDOM_SEED = 1

for demand in ('peak', 'off_peak'):
    # heuristic
    for i in range(len(alphas)):
        scenario_name = f'heuristic_{float_to_str(alphas[i])}'
        np.random.seed(RANDOM_SEED)
        tmp_history = evaluate_heuristic(
            alpha=alphas[i], beta=HEURISTIC_BETA, n_episodes=N_EPISODES, 
            output_history=True,
            scenario_name=scenario_name, demand_scenario=demand)
        # for ky in tmp_history:
        #     if ky not in main_history:
        #         main_history[ky] = []
        #     tmp_history[ky]['demand'] = demand
        #     main_history[ky].append(tmp_history[ky])

Vehicle 1 at stop 1 with expected headway 91
Time difference: 59
Current time: 10887, Old depart time 10886.8 New depart time: 10945.6
Vehicle 0 at stop 3 with expected headway 83
Time difference: 67
Current time: 4974, Old depart time 4974.0 New depart time: 5041.2
Vehicle 0 at stop 0 with expected headway 124
Time difference: 26
Current time: 6031, Old depart time 6043.100000000002 New depart time: 6068.999999999999
Vehicle 0 at stop 1 with expected headway 116
Time difference: 34
Current time: 6165, Old depart time 6177.099999999999 New depart time: 6211.299999999999
Vehicle 0 at stop 3 with expected headway 80
Time difference: 70
Current time: 6271, Old depart time 6271.299999999999 New depart time: 6341.299999999999
Vehicle 0 at stop 1 with expected headway 142
Time difference: 8
Current time: 6682, Old depart time 6700.5 New depart time: 6708.2
Vehicle 0 at stop 5 with expected headway 146
Time difference: 4
Current time: 8218, Old depart time 8238.400000000001 New depart time: 8